# 1.2 and 1.3 Data Cleaning for train Data

In [88]:
import pandas as pd
import numpy as np
trainData = pd.read_csv('dataset/cases_train.csv')
testData = pd.read_csv('dataset/cases_test.csv')

In [89]:
#Remove outlier of NaN rows
trainData[trainData['latitude'].isnull()]

,age,sex,province,country,latitude,longitude,date_confirmation,additional_information,source,outcome
26410,NaN,female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nonhospitalized
230376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nonhospitalized


In [90]:
trainData = trainData.drop([26410,230376]).reset_index()

# Cleaning age and binning

In [91]:
changingToNaN = ['30-39','0-9','10-19','21-39','65-','13-19','22-80','21-61','13-69','18-65','8-68','33-78','0-19','20-39','60-','25-59','19-77','30-70','20-57','21-72','2-87','18-','11-80','18-99','50-69','18-50','40-69','16-80','28-35','0-18','13-65','40-89','34-66','9-69','80-','20-69','23-72','23-84','38-68','22-60','50-100','41-60','27-40','27-58','18-60','30-40','18 - 100','18-49','50-99','0-60','39-77','19-65','34-44','19-75','20-70','15-88','4-64','17-66','17-65','30-35','0-10']
#these are the values that simply do not work with our binning strategies, therefore for analysis we will change these values to NaN instead
for i in changingToNaN:
    trainData['age'] = trainData['age'].replace({i:np.NaN})

trainData['age'] = trainData['age'].replace({'80+':'80'})   
trainData['age'] = trainData['age'].replace({'90+':'90'})
trainData['age'] = trainData['age'].replace({'85+':'85'})
trainData['age'] = trainData['age'].replace({'8 month':'0.67'})
trainData['age'] = trainData['age'].replace({'5 month':'0.42'})

In [92]:
i = 0
while i < len(trainData['age']): #iterating throuh database
    if type(trainData['age'][i]) == str: #skip NaN values
        x = trainData['age'][i]
        x = x.split('-') #split the string on '-' 
        if len(x) == 2: #these are the values that are a range
            trainData['age'] = trainData['age'].replace({trainData['age'][i]:x[0]}) #replace the range with simply the lower bound of range (this will be fine as we are binning on the whole range of all these values)
    
    i += 1

In [93]:
#because we are binning with on [0,4],[5,14],[15,34],[35,59],[60,79],[80+] (notice they are all tight boundaries) therefore we dont care about any values that are decimal and we can simply make all values int

trainData['age'] = pd.to_numeric(trainData['age'],downcast='integer')

In [94]:
bins = [-0.1, 4, 14, 34, 59, 79, 150]
trainData['binned'] =  np.searchsorted(bins, trainData['age'].values)

# Impute 'Unknown' for missing sex, province, additional_information, and source

In [95]:
trainData['sex'] = trainData['sex'].fillna('Unknown')
trainData['province'] = trainData['province'].fillna('Unknown')
trainData['additional_information'] = trainData['additional_information'].fillna('Unknown')
trainData['source'] = trainData['source'].fillna('Unknown')

#Only rows with missing country had a province of Taiwan so fill country column with Taiwan
trainData['country'] = trainData['country'].fillna('Taiwan')

# Impute mean date for date column and clean dates that give a range

In [96]:
def split_date(datestr):
    if datestr == np.nan:
        return np.nan
    elif '-' in str(datestr):
        
        div = datestr.split('-')[0] #take the first date
#         print(f'the string was {datestr} the split is {div}')
        return div
    else:
        return datestr


trainData['date_confirmation'] = trainData['date_confirmation'].apply(split_date)
meandate = pd.to_datetime(trainData.date_confirmation).mean().date()
the_date = str(meandate.day) + '.' + str(meandate.month) + '.' + str(meandate.year)
trainData['date_confirmation'] = trainData['date_confirmation'].fillna(the_date) # fill date with mean date
trainData['date_confirmation'] = pd.to_datetime(trainData['date_confirmation'].apply(lambda x: str.strip(x)))

In [97]:
trainData.head()

,index,age,sex,province,country,latitude,longitude,date_confirmation,additional_information,source,outcome,binned
0,0,NaN,Unknown,Delhi,India,28.614740,77.209100,2020-05-26,Unknown,https://twitter.com/CMODelhi/status/1265203060...,recovered,7
1,1,NaN,Unknown,Uttar Pradesh,India,25.436090,81.847180,2020-05-20,Unknown,https://t.me/indiacovid/5222,hospitalized,7
2,2,NaN,Unknown,Maharashtra,India,18.504220,73.853020,2020-05-26,Unknown,https://t.me/indiacovid/5601,hospitalized,7
3,3,15.0,female,Baden-Wurttemberg,Germany,48.120860,7.849500,2020-03-15,Unknown,Unknown,nonhospitalized,3
4,4,NaN,Unknown,Gujarat,India,23.188409,73.647122,2020-05-20,Unknown,https://twitter.com/PIBAhmedabad/status/126311...,hospitalized,7


# Data cleaning for test data

# Impute 'Unknown' for Sex, Province, additional_information, source

In [73]:
testData.head()

,age,sex,province,country,latitude,longitude,date_confirmation,additional_information,source,outcome
0,NaN,NaN,Rajasthan,India,24.587000,73.69848,11.05.2020,NaN,https://t.me/indiacovid/4565,NaN
1,15-34,male,Bayern,Germany,49.993767,9.66288,09.04.2020,NaN,NaN,NaN
2,NaN,NaN,Maharashtra,India,19.200000,72.96667,23.05.2020,NaN,https://phdmah.maps.arcgis.com/apps/opsdashboa...,NaN
3,NaN,NaN,Madhya Pradesh,India,23.180870,75.78816,22.05.2020,NaN,https://twitter.com/JansamparkMP/status/126384...,NaN
4,56,female,Lima,Peru,-12.003950,-77.00695,19.04.2020,NaN,NaN,NaN


In [79]:
testData['sex'] = testData['sex'].fillna('Unknown')
testData['province'] = testData['province'].fillna('Unknown')
testData['additional_information'] = testData['additional_information'].fillna('Unknown')
testData['source'] = testData['source'].fillna('Unknown')

# Fixing date, same as above

In [75]:
testData['date_confirmation'] = testData['date_confirmation'].apply(split_date)
meandate = pd.to_datetime(testData.date_confirmation).mean().date()
the_date = str(meandate.day) + '.' + str(meandate.month) + '.' + str(meandate.year)
testData['date_confirmation'] = testData['date_confirmation'].fillna(the_date) # fill date with mean date
testData['date_confirmation'] = pd.to_datetime(testData['date_confirmation'].apply(lambda x: str.strip(x)))

# Fixing age, same as above

In [67]:
changingToNaN = ['30-39','0-9','10-19','21-39','65-','13-19','22-80','21-61','13-69','18-65','8-68','33-78','0-19','20-39','60-','25-59','19-77','30-70','20-57','21-72','2-87','18-','11-80','18-99','50-69','18-50','40-69','16-80','28-35','0-18','13-65','40-89','34-66','9-69','80-','20-69','23-72','23-84','38-68','22-60','50-100','41-60','27-40','27-58','18-60','30-40','18 - 100','18-49','50-99','0-60','39-77','19-65','34-44','19-75','20-70','15-88','4-64','17-66','17-65','30-35','0-10']
#these are the values that simply do not work with our binning strategies, therefore for analysis we will change these values to NaN instead
for i in changingToNaN:
    trainData['age'] = trainData['age'].replace({i:np.NaN})

testData['age'] = testData['age'].replace({'80+':'80'})   
testData['age'] = testData['age'].replace({'90+':'90'})
testData['age'] = testData['age'].replace({'85+':'85'})
testData['age'] = testData['age'].replace({'8 month':'0.67'})
testData['age'] = testData['age'].replace({'5 month':'0.42'})

In [ ]:
i = 0
while i < len(trainData['age']): #iterating throuh database
    if type(trainData['age'][i]) == str: #skip NaN values
        x = trainData['age'][i]
        x = x.split('-') #split the string on '-' 
        if len(x) == 2: #these are the values that are a range
            trainData['age'] = trainData['age'].replace({trainData['age'][i]:x[0]}) #replace the range with simply the lower bound of range (this will be fine as we are binning on the whole range of all these values)
    
    i += 1

# 1.4: Aggregate the data to the State level for USA

In [83]:
df_location = pd.read_csv('dataset/location.csv')
df_US = df_location[df_location['Country_Region'] == 'US']
df_US.head()

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
631,Alabama,US,2020-09-20 04:22:56,32.539527,-86.644082,1673,24,0,1649.0,"Autauga, Alabama, US",2994.505003,1.434549
632,Alabama,US,2020-09-20 04:22:56,30.727750,-87.722071,5047,49,0,4998.0,"Baldwin, Alabama, US",2260.856321,0.970874
633,Alabama,US,2020-09-20 04:22:56,31.868263,-85.387129,830,7,0,823.0,"Barbour, Alabama, US",3362.229604,0.843373
634,Alabama,US,2020-09-20 04:22:56,32.996421,-87.125115,628,10,0,618.0,"Bibb, Alabama, US",2804.322586,1.592357
635,Alabama,US,2020-09-20 04:22:56,33.982109,-86.567906,1542,14,0,1528.0,"Blount, Alabama, US",2666.620551,0.907912


In [84]:
#Group by state
df_state = df_US.groupby(['Country_Region','Province_State']).agg({
    'Lat':'first', #simply take the first lat/lon you see and use it for the state
    'Long_':'first',
    'Confirmed':'sum',
    'Deaths':'sum',
    'Recovered':'sum',
    'Active':'sum',
    'Incidence_Rate':'mean', #gets fixed below
    'Case-Fatality_Ratio':'sum' #gets fixed below
}).reset_index(level=0, drop=True) #get rid of USA in index
df_state['Case-Fatality_Ratio'] = df_state['Confirmed']/df_state['Deaths'] #recalculate case-fatality ratio
df_state = df_state.drop(['Diamond Princess','Grand Princess','Recovered'])
df_state.head()

,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incidence_Rate,Case-Fatality_Ratio
Province_State,,,,,,,,
Alabama,32.539527,-86.644082,144164,2437,0,141727.0,3239.932554,59.156340
Alaska,55.322224,-161.972202,6729,45,0,6684.0,681.806753,149.533333
Arizona,35.394650,-109.489238,212942,5451,0,207491.0,2927.250679,39.064759
Arkansas,34.291452,-91.372773,75160,1181,0,73979.0,2564.972414,63.640982
California,37.646294,-121.892927,783313,15018,0,768295.0,1544.858420,52.158277


In [85]:
#EXTERNAL SOURCE population data of each state
df_pop = pd.read_csv('dataset/State Populations.csv').rename({'State':'Province_State'},axis=1).set_index('Province_State')
df_state = df_state.join(df_pop)
df_state.head()

,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incidence_Rate,Case-Fatality_Ratio,2018 Population
Province_State,,,,,,,,,
Alabama,32.539527,-86.644082,144164,2437,0,141727.0,3239.932554,59.156340,4888949
Alaska,55.322224,-161.972202,6729,45,0,6684.0,681.806753,149.533333,738068
Arizona,35.394650,-109.489238,212942,5451,0,207491.0,2927.250679,39.064759,7123898
Arkansas,34.291452,-91.372773,75160,1181,0,73979.0,2564.972414,63.640982,3020327
California,37.646294,-121.892927,783313,15018,0,768295.0,1544.858420,52.158277,39776830


In [86]:
df_state['Incidence_Rate'] = (df_state['Confirmed'] / (df_state['2018 Population']/100000))
df_state = df_state.drop('2018 Population',axis=1)
df_state

,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incidence_Rate,Case-Fatality_Ratio
Province_State,,,,,,,,
Alabama,32.539527,-86.644082,144164,2437,0,141727.0,2948.772834,59.156340
Alaska,55.322224,-161.972202,6729,45,0,6684.0,911.704613,149.533333
Arizona,35.394650,-109.489238,212942,5451,0,207491.0,2989.121967,39.064759
Arkansas,34.291452,-91.372773,75160,1181,0,73979.0,2488.472275,63.640982
California,37.646294,-121.892927,783313,15018,0,768295.0,1969.269547,52.158277
Colorado,39.874321,-104.336258,64336,2013,0,62323.0,1131.838536,31.960258
Connecticut,41.268099,-73.388117,55527,4492,0,51035.0,1547.280716,12.361309
Delaware,39.086466,-75.568849,19449,621,0,18828.0,2002.615375,31.318841
District of Columbia,38.904178,-77.016560,14902,619,0,14283.0,2117.940672,24.074313
